In [ ]:
from astropy import units as u
from cartopy import crs as ccrs
from eigenroutines import cut_to_eclipse
from matplotlib import pyplot as plt
import numpy as np
from os import mkdir, path
import starry
from system_properties import warm_Jupiter as system_properties
from visibility import visibility_kernel
from warnings import filterwarnings

%matplotlib inline

starry.config.lazy = False
starry.config.quiet = True

filterwarnings("ignore", category=FutureWarning)

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
time_extent = "T1-T4"

In [ ]:
star = starry.Primary(starry.Map(udeg=2), **system_properties["primary"])
star.map[1:] = [0.5, 0.25]
planet = starry.Secondary(starry.Map(ydeg=max_degree,
                                     **system_properties["secondary map"]),
                          **system_properties["secondary"])
system = starry.System(star, planet)

In [ ]:
times = cut_to_eclipse(system=system,
                       time_resolution=time_resolution,
                       extent=time_extent)

In [ ]:
lat = np.linspace(-90, 90, 181)
lon = np.linspace(-180, 180, 361)

lons, lats = np.meshgrid(lon, lat)
lats = np.deg2rad(lats) * u.rad
lons = np.deg2rad(lons) * u.rad

In [ ]:
obl_LOS = 60
obl_insky = 0
visibility = visibility_kernel(system=system, times=times,
                               lat=lats, lon=lons,
                               rotation_rate= 1.6 * (2*np.pi*u.rad)/(planet.porb*u.day),
                               obl_LOS=obl_LOS, obl_insky=obl_insky)

In [ ]:
plt.imshow(visibility[1], aspect="equal", extent=(-180, 180, -90, 90))
# plt.savefig("visibility_test.pdf", bbox_inches="tight", dpi=300)

In [ ]:
plt.figure(figsize=(12, 6))
for i in range(20):
    ax = plt.subplot(4, 5, i+1, projection=ccrs.Orthographic(central_latitude=obl_LOS,
                                                             central_longitude=270))
    ax.imshow(visibility[(i+0)],
              aspect="equal",
              extent=(-180, 180, -90, 90),
              cmap=plt.get_cmap("magma"),
              transform=ccrs.PlateCarree())
plt.savefig("grid-vis.pdf", bbox_inches="tight")

In [ ]:
halfecl = np.shape(visibility)[0] // 2

In [ ]:
ingress_vis = np.sum(visibility[:halfecl]**2, axis=0)
egress_vis = np.sum(visibility[halfecl:]**2, axis=0)
full_vis = np.sum(visibility**2, axis=0)

In [ ]:
sum_ax = plt.axes(projection=ccrs.Orthographic(central_latitude=90, central_longitude=260))
sum_ax.imshow(full_vis,
              aspect="equal",
              extent=(-180, 180, -90, 90),
              cmap=plt.get_cmap("magma"),
              transform=ccrs.PlateCarree())

In [ ]:
plt.imshow(ingress_vis, aspect="equal",
           extent=(-180, 180, -90, 90),
           cmap=plt.get_cmap("magma"),
           vmin=0, vmax=np.max(ingress_vis))

In [ ]:
plt.imshow(egress_vis, aspect="equal",
           extent=(-180, 180, -90, 90),
           cmap=plt.get_cmap("magma"),
           vmin=0, vmax=np.max(ingress_vis))

In [ ]:
plt.imshow(full_vis, aspect="equal",
           extent=(-180, 180, -90, 90),
           cmap=plt.get_cmap("magma"),
           vmin=0, vmax=np.max(ingress_vis))